In [1]:
from edgar_functions import *

ticker = "WSM"

In [2]:
accession_num_series = get_10K_accessionNumbers_for_ticker(ticker)

In [3]:
accession_num_series

reportDate
2023-01-29    000162828023009175
2022-01-30    000162828022007494
2021-01-31    000119312521100319
2020-02-02    000119312520088937
2019-02-03    000119312519097973
2018-01-28    000119312518102232
2017-01-29    000119312517104341
2016-01-31    000119312516525847
2015-02-01    000119312515118009
2014-02-02    000119312514129974
2013-02-03    000119312513142309
2012-01-29    000119312512140070
2011-01-30    000119312511085046
2010-01-31    000119312510075151
Name: 10K_accessionNumber, dtype: object

In [4]:
soup = get_statement_soup(
    ticker, accession_num_series[-3], "consolidated balance sheets"
)

In [5]:
soup

<html><body><document>
<type>XML
<sequence>54
<filename>R3.htm
<description>IDEA: XBRL DOCUMENT
<text>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="report.css" rel="StyleSheet" type="text/css"/><script src="Show.js" type="text/javascript">/* Do Not Remove This Comment */</script>
<span style="display: none;">v2.4.0.6</span><table border="0" cellspacing="2" class="report" id="ID0ETSAG">
<tr>
<th class="tl" colspan="1" rowspan="1">
<div style="width: 200px;"><strong>Consolidated Balance Sheets (USD $)<br/></strong></div>
</th>
<th class="th" colspan="2">
<div>Jan. 29, 2012</div>
</th>
<th class="th" colspan="2">
<div>Jan. 30, 2011</div>
</th>
</tr>
<tr class="re">
<td class="pl" style="border-bottom: 0px;" valign="top"><a class="a" href="javascript:void(0);" onclick="top.Show.showAR( this, 'defref_us-gaap_AssetsCurrentAbstract', window );"><strong>Current assets</strong></a></td>
<td class="text"> <span></span></td>
<td class="fn" style="border-bottom:

In [6]:
def extract_columns_and_values_from_statement(soup: BeautifulSoup):
    in_thousands = False
    for th in soup.find_all("th"):
        if re.search(r"in Thousands", th.get_text(), re.IGNORECASE):
            in_thousands = True
            break

    columns = []
    values_set = []
    for row in soup.select("tr.re, tr.ro, tr.reu, tr.rou"):
        onclick_attr = row.select_one("td.pl a, td.pl.custom a")["onclick"]
        column_title = onclick_attr.split("defref_")[-1].split("',")[0]
        columns.append(column_title)
        values = [np.NaN, np.NaN]

        for i, cell in enumerate(row.select("td.text, td.nump, td.num")):
            if "text" in cell.get("class"):
                continue
            value = (
                cell.text.replace("$", "")
                .replace(",", "")
                .replace("(", "")
                .replace(")", "")
                .strip()
            )

            value = keep_numbers_and_decimals_only_in_string(value)
            if value:
                value = float(value)
                if not in_thousands:
                    value /= 1000

            if "nump" in cell.get("class"):
                values[i] = value
            else:
                values[i] = -value if value else value

        values_set.append(values)

    return columns, values_set

In [7]:
columns, values = extract_columns_and_values_from_statement(soup)
values

[[nan, nan],
 [502757.0, 628403.0],
 [14732.0, 12512.0],
 [45961.0, 41565.0],
 [553461.0, 513381.0],
 [34294.0, 36825.0],
 [24188.0, 21120.0],
 [91744.0, 85612.0],
 [9229.0, 8176.0],
 [1276366.0, 1347594.0],
 [734672.0, 730556.0],
 [12382.0, 32646.0],
 [37418.0, 20966.0],
 [2060838.0, 2131762.0],
 [nan, nan],
 [218329.0, 227963.0],
 [111774.0, 122440.0],
 [190417.0, 192450.0],
 [22435.0, 41997.0],
 [1795.0, 1542.0],
 [27049.0, 25324.0],
 [571799.0, 611716.0],
 [181762.0, 202135.0],
 [5478.0, 7130.0],
 [46537.0, 51918.0],
 [805576.0, 872899.0],
 [nan, nan],
 [nan, nan],
 [0.0, 0.0],
 [1005.0, 1049.0],
 [478720.0, 466885.0],
 [762947.0, 777939.0],
 [12590.0, 12990.0],
 [1255262.0, 1258863.0],
 [2060838.0, 2131762.0]]

In [8]:
dates = get_datetime_index_dates_from_statement(soup)

In [9]:
columns, values = extract_columns_and_values_from_statement(soup)

In [10]:
process_one_statement(ticker, accession_num_series[7], "consolidated balance sheets").T

,2016-01-31,2015-02-01
us-gaap_AssetsCurrentAbstract,NaN,NaN
us-gaap_CashAndCashEquivalentsAtCarryingValue,193647.0,222927.0
us-gaap_ReceivablesNetCurrent,79304.0,67465.0
us-gaap_InventoryNet,978138.0,887701.0
us-gaap_PrepaidAdvertising,28919.0,33942.0
us-gaap_PrepaidExpenseCurrent,44654.0,36265.0
us-gaap_DeferredTaxAssetsNetCurrent,NaN,130618.0
us-gaap_OtherAssetsCurrent,11438.0,13005.0
us-gaap_AssetsCurrent,1336100.0,1391923.0
us-gaap_PropertyPlantAndEquipmentNet,886813.0,883012.0


In [11]:
form_statement_for_all_available_years(ticker, "consolidated balance sheets").T

consolidated balance sheets for accession number: 000162828023009175 has been processed.
consolidated balance sheets for accession number: 000162828022007494 has been processed.
consolidated balance sheets for accession number: 000119312521100319 has been processed.
consolidated balance sheets for accession number: 000119312520088937 has been processed.
consolidated balance sheets for accession number: 000119312519097973 has been processed.
consolidated balance sheets for accession number: 000119312518102232 has been processed.
consolidated balance sheets for accession number: 000119312517104341 has been processed.
consolidated balance sheets for accession number: 000119312516525847 has been processed.
consolidated balance sheets for accession number: 000119312515118009 has been processed.
consolidated balance sheets for accession number: 000119312514129974 has been processed.
consolidated balance sheets for accession number: 000119312513142309 has been processed.
consolidated balance 

consolidated balance sheets for accession number: 000119312511085046 has been processed.


ERROR:root:Failed to get statement soup: 'consolidated balance sheets' for accession number: 000119312510075151


consolidated balance sheets for accession number: 000119312510075151 has been processed.


,2023-01-29,2022-01-30,2021-01-31,2020-02-02,2019-02-03,2018-01-28,2017-01-29,2016-01-31,2015-02-01,2014-02-02,2013-02-03,2012-01-29,2011-01-30
us-gaap_AssetsCurrentAbstract,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
us-gaap_CashAndCashEquivalentsAtCarryingValue,367344.0,850338.0,1200337.0,432162.0,338954.0,390136.0,213713.0,193647.0,222927.0,330121.0,424555.0,502757.0,628403.0
us-gaap_ReceivablesNetCurrent,115685.0,131683.0,143728.0,111737.0,107102.0,90119.0,88803.0,79304.0,67465.0,60330.0,62985.0,45961.0,41565.0
us-gaap_InventoryNet,1456123.0,1246372.0,1006299.0,1100544.0,1124992.0,1061593.0,977505.0,978138.0,887701.0,813160.0,640024.0,553461.0,513381.0
us-gaap_PrepaidExpenseCurrent,64961.0,69252.0,93822.0,90426.0,101356.0,62204.0,52882.0,44654.0,36265.0,35309.0,26339.0,24188.0,21120.0
us-gaap_OtherAssetsCurrent,31967.0,26249.0,22894.0,20766.0,21939.0,11876.0,10652.0,11438.0,13005.0,10852.0,9819.0,9229.0,8176.0
us-gaap_AssetsCurrent,2036080.0,2323894.0,2467080.0,1755635.0,1694343.0,1636445.0,1367180.0,1336100.0,1391923.0,1419103.0,1316772.0,1276366.0,1347594.0
us-gaap_PropertyPlantAndEquipmentNet,1065381.0,920773.0,873894.0,929038.0,929635.0,932283.0,923283.0,886813.0,883012.0,849293.0,812037.0,734672.0,730556.0
us-gaap_OperatingLeaseRightOfUseAsset,1286452.0,1132764.0,1086009.0,1166383.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
us-gaap_DeferredIncomeTaxAssetsNet,81389.0,56585.0,61854.0,47977.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
